# Cycle GAN

In [1]:
import os
import sys
import glob
import random
import itertools
from PIL import Image

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [2]:
DATASET_PATH = os.path.join('..', 'Datasets', 'summer2winter_yosemite')

In [3]:
sys.path.append("../functions/")

In [4]:
from utils import ReplayBuffer

In [5]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        
        conv_block = [
            # Better padding that zero padding to conserve image's distribution
            nn.ReflectionPad2d(1), 
            nn.Conv2d(in_features, in_features, 3),
            # Not as good in normalization as BN, but it is better conserving the contrast
            nn.InstanceNorm2d(in_features),
            nn.ReLU(True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
        ]
        self.conv_block = nn.Sequential(*conv_block)

    def forward(self, x):
        return self.conv_block(x) + x

In [6]:
class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_residual_blocks=9):
        super(Generator, self).__init__()

        model = [
            nn.ReflectionPad2d(3),
            nn.Conv2d(input_nc, 64, 7),
            nn.InstanceNorm2d(64),
            nn.ReLU(True),
        ]

        in_features = 64
        out_features = in_features * 2

        # Encoding
        for _ in range(2):
            model += [
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(True),
            ]
            in_features = out_features
            out_features = in_features * 2

        # Residual transformations
        for _ in range(n_residual_blocks):
            model += [ResidualBlock(in_features)]

        # Decoding
        out_features = in_features // 2
        for _ in range(2):
            model += [
                nn.ConvTranspose2d(
                    in_features, out_features, 3, stride=2, padding=1, output_padding=1
                ),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(True),
            ]
            in_features = out_features
            out_features = in_features // 2

        model += [nn.ReflectionPad2d(3), nn.Conv2d(64, output_nc, 7), nn.Tanh()]

        self.model = nn.Sequential(*model)
    
    def forward(self, x):
        return self.model(x)


In [7]:
class Discrimator(nn.Module):
    """This will replicate the PatchGAN. Discriminates style or texture
    """
    
    def __init__(self, input_nc):
        super(Discrimator, self).__init__()
        
        model = [
            nn.Conv2d(input_nc, 64, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, padding=1),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, padding=1)
        ]
        
        self.model = nn.Sequential(*model)
    
    def forward(self, x):
        x = self.model(x)
        return F.avg_pool2d(x, x.size()[2:]).view(x.size()[0], -1)

In [8]:
class ImageDataset(Dataset):
    
    def __init__(self, base_dir, transform=None, split='train'):
        self.base_dir = base_dir
        self.transform = transforms.Compose(transform)
        self.files_A = sorted(glob.glob(os.path.join(self.base_dir, f"{split}/A/*.*")))
        self.files_B = sorted(glob.glob(os.path.join(self.base_dir, f"{split}/B/*.*")))
    
    def test(self):
        print(self.base_dir)
        print(f"Files A: {self.files_A} && Files B: {self.files_B}")
        
    def __len__(self):
        return max(len(self.files_A), len(self.files_B))
    
    def __getitem__(self, idx):
        image_A = self.transform(Image.open(self.files_A[idx]))
        image_B = self.transform(Image.open(self.files_B[random.randint(0, len(self.files_B)-1)]))

        return {"A": image_A, "B":image_B}

In [9]:
epoch = 0 
n_epochs = 200
batch_size = 32
lr = 2e-4
size = 256
input_nc = 3
output_nc = 3
decay_epoch = 100

gpu = True
n_cpu = 10
base_dir = '../Datasets/summer2winter_yosemite/'

DEVICE = torch.device("mps" if gpu else "cpu")

In [10]:
class LambdaLR():
    
    def __init__(self, n_epochs, offset, decay_start_epoch):
        assert ((n_epochs - decay_start_epoch) > 0)
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch
    
    def step(self, epoch):
        return 1 - max(0, epoch + self.offset - self.decay_start_epoch) / (self.n_epochs - self.decay_start_epoch)
        

In [11]:
def weights_init_normal(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.normal(m.weight.data, 0.0, 0.02)
    elif isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant(m.bias, 0.0)


netG_A2B = Generator(input_nc, output_nc)
netG_B2A = Generator(input_nc, output_nc)
netD_A = Discrimator(input_nc)
netD_B = Discrimator(input_nc)

netG_A2B.apply(weights_init_normal)
netG_B2A.apply(weights_init_normal)
netD_A.apply(weights_init_normal)
netD_B.apply(weights_init_normal)

if gpu:
    netG_A2B.to(DEVICE)
    netG_B2A.to(DEVICE)
    netD_A.to(DEVICE)
    netD_B.to(DEVICE)

criterion_GAN = nn.MSELoss()
criterion_cycle = nn.L1Loss()
criterion_identity = nn.L1Loss()

optimizer_G = torch.optim.Adam(
    itertools.chain(netG_A2B.parameters(), netG_B2A.parameters()),
    lr=lr,
    betas=(0.5, 0.999),
)
optimizer_D_A = torch.optim.Adam(netD_A.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D_B = torch.optim.Adam(netD_B.parameters(), lr=lr, betas=(0.5, 0.999))

# Learning rate schedulers


lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=LambdaLR(n_epochs, epoch, decay_epoch).step)
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, lr_lambda=LambdaLR(n_epochs, epoch, decay_epoch).step)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, lr_lambda=LambdaLR(n_epochs, epoch, decay_epoch).step)

/var/folders/bs/jxzw14n53ksgt9_q98rq0_0m0000gn/T/ipykernel_3224/385112892.py:3: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(m.weight.data, 0.0, 0.02)


In [12]:
# inputs y targets
tensor = torch.tensor
target_real = tensor(batch_size).fill_(1.0)
target_fake = tensor(batch_size).fill_(0.0)

fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

# Dataloader
transform = [
    transforms.Resize(int(size * 1.12), Image.BICUBIC),
    transforms.RandomCrop(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]

dataloader = DataLoader(
    ImageDataset(base_dir, transform=transform),
    batch_size=batch_size,
    shuffle=True,
    num_workers=n_cpu,
    drop_last=True,
)


/var/folders/bs/jxzw14n53ksgt9_q98rq0_0m0000gn/T/ipykernel_3224/3583057090.py:11: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  transforms.Resize(int(size * 1.12), Image.BICUBIC),
/Users/mmenendezg/Developer/platzi/.venv/lib/python3.10/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [13]:
def Gen_GAN_loss(G, D, real, loss, target_real):
    fake = G(real)
    pred_fake = D(fake)
    L = loss(pred_fake, target_real)
    return L, fake

def cycle_loss(G1, G2, real, loss):
    recovered = G2(G1(real))
    L = loss(recovered)
    return L

def identity_loss(G, real, loss):
    same = G(real)
    L = loss(same, real)
    return L

def Disc_GAN_loss(D2, fake2, real2, fake_2_buffer, loss, target_real, target_fake):
    pred_real = D2(real2)
    loss_D2_real = loss(pred_real, target_real)

    fake2 = fake_2_buffer.push_and_pop(fake2)
    pred_fake = D2(fake2.detach())
    loss_D2_fake = loss(pred_fake, target_fake)
    loss_D2 = (loss_D2_real + loss_D2_fake) * 0.5
    return loss_D2

In [15]:
for epoch in range(epoch, n_epochs):
    for i, batch in enumerate(dataloader):
        real_A = batch['A'].to(device)
        real_B = batch['B'].to(device)
        
        # Generativas
        optimizer_G.zero_grad()
        
        loss_GAN_A2B, fake_B = Gen_GAN_loss(netG_A2B, netD_B, real_A, criterion_GAN, target_real)
        loss_GAN_B2A, fake_A = Gen_GAN_loss(netG_B2A, netD_A, real_B, criterion_GAN, target_real)
        
        loss_cycle_ABA = cycle_loss(netG_A2B, netG_B2A, real_A, criterion_cycle)
        loss_cycle_BAB = cycle_loss(netG_B2A, netG_A2B, real_B, criterion_cycle)
        
        loss_identity_A = identity_loss(netG_B2A, real_A, criterion_identity)
        loss_identity_B = identity_loss(netG_A2B, real_B, criterion_identity)
        
        loss_G = (loss_GAN_A2B + loss_GAN_B2A) + 10.0*(loss_cycle_ABA + loss_cycle_BAB) + 5.0 *(loss_identity_A + loss_identity_B)
        loss_G.backward()
        
        optimizer_G.step
        
        #Discriminativas
        optimizer_D_A.zero_grad()
        
        loss_D_A = Disc_GAN_loss(netD_A, fake_A, real_A, fake_A_buffer, criterion_GAN, target_real, target_fake)
        loss_D_A.backward()
        optimizer_D_A.step()
        
        optimizer_D_B.zero_grad()
        
        loss_D_B = Disc_GAN_loss(netD_B, fake_B, real_B, fake_B_buffer, criterion_GAN, target_real, target_fake)
        loss_D_B.backward()
        optimizer_D_B.step()
    
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    lr_scheduler_D_B.step()

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'ImageDataset' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pi

RuntimeError: DataLoader worker (pid(s) 3308) exited unexpectedly

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'ImageDataset' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pi